##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# TFRecord e tf.train.Example

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/tutorials/load_data/tfrecord"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/tutorials/load_data/tfrecord.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

O TFRecord é um formato simples para armazenar uma sequência de registros binários.

Os [buffers de protocolo](https://developers.google.com/protocol-buffers/) são uma biblioteca de várias plataformas e linguagens para a serialização eficiente de dados estruturados.

As mensagens de protocolo são definidas por arquivos `.proto` e geralmente são a maneira mais fácil de entender um tipo de mensagem.

`tf.train.Example` (ou protobuf) é um tipo de mensagem flexível que representa um mapeamento `{"string": value}`. Foi criado para o TensorFlow e é usado em APIs de alto nível, como o [TFX](https://www.tensorflow.org/tfx/).

Este notebook mostra como criar, processar e usar a mensagem `tf.train.Example` e depois serializar, escrever e ler mensagens `tf.train.Example` de e para arquivos `.tfrecord`.

Observação: embora sejam úteis, essas estruturas são opcionais. Não há necessidade de converter código existente para usar TFRecords, a menos que você esteja [usando o tf.data](https://www.tensorflow.org/guide/data) e a leitura dos dados ainda seja o gargalo do treinamento. Consulte dicas de desempenho do dataset no guia [Melhor desempenho com a API tf.data](https://www.tensorflow.org/guide/data_performance).

Observação: em geral, você deve fragmentar seus dados em vários arquivos para paralelizar a I/O (em um ou mais hosts). A regra geral é ter pelo menos 10 vezes mais arquivos que o número de hosts lendo dados. Ao mesmo tempo, cada arquivo deve ser grande o suficiente (no mínimo 10 MB e de preferência maior que 100 MB) para você aproveitar a pré-busca de I/O. Por exemplo, digamos que você tenha `X` GB de dados e planeje treinar em até `N` hosts. O ideal é compartilhar os dados em ~`10*N` arquivos, desde que ~`X/(10*N)` seja maior que 10 MB (preferencialmente, maior que 100 MB). Se for menor que isso, talvez seja necessário criar menos fragmentos para equilibrar os benefícios do paralelismo e os benefícios da pré-busca de I/O.

## Configuração

In [ ]:
import tensorflow as tf

import numpy as np
import IPython.display as display

## `tf.train.Example`

### Tipos de dados para `tf.train.Example`

Essencialmente, o `tf.train.Example` é um mapeamento de `{"string": tf.train.Feature}`.

O tipo de mensagem `tf.train.Feature` aceita um dos três tipos a seguir (Consulte o [arquivo `.proto`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto) para referência). A maioria dos outros tipos genéricos pode ser coagida em um destes:

1. `tf.train.BytesList` (os seguintes tipos podem ser coagidos)

- `string`
- `byte`

1. `tf.train.FloatList` (os seguintes tipos podem ser coagidos)

- `float` (`float32`)
- `double` (`float64`)

1. `tf.train.Int64List` (os seguintes tipos podem ser coagidos)

- `bool`
- `enum`
- `int32`
- `uint32`
- `int64`
- `uint64`

Para converter um tipo padrão do TensorFlow em um `tf.train.Feature` compatível com `tf.train.Example`, use as funções de atalho abaixo. Observe que cada função aceita um valor de entrada escalar e retorna um `tf.train.Feature` com um dos três tipos `list` acima:

In [ ]:
# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

Observação: para simplicidade, este exemplo só usa entradas escalares. A maneira mais simples de lidar com características não escalares é usar `tf.io.serialize_tensor` para converter tensores em strings binárias. As strings são escalares no TensorFlow. Use `tf.io.parse_tensor` para converter a string binária de volta para um tensor.

Confira abaixo alguns exemplos de como essas funções funcionam. Observe os tipos de entrada variáveis e os tipos de saída padronizados. Se o tipo de entrada para uma função não corresponder a um dos tipos que podem ser coagidos indicados acima, a função lançará uma exceção (por exemplo, `_int64_feature(1.0)` gerará um erro porque `1.0` é um float — portanto, deve ser usado com a função `_float_feature`):

In [ ]:
print(_bytes_feature(b'test_string'))
print(_bytes_feature(u'test_bytes'.encode('utf-8')))

print(_float_feature(np.exp(1)))

print(_int64_feature(True))
print(_int64_feature(1))

Todas as mensagens proto podem ser serializadas para uma string binária usando o método `.SerializeToString`:

In [ ]:
feature = _float_feature(np.exp(1))

feature.SerializeToString()

### Criando uma mensagem `tf.train.Example`

Digamos que você queira criar uma mensagem `tf.train.Example` a partir de dados existentes. Na prática, o dataset pode vir de qualquer lugar, mas o procedimento de criar a mensagem `tf.train.Example` a partir de uma única observação será o mesmo:

1. Em cada observação, cada valor precisa ser convertido em `tf.train.Feature` com um dos 3 tipos compatíveis, usando uma das funções acima.

2. Você cria um mapa (dicionário) a partir da string de nome da característica para o valor da característica codificado produzido no nº 1.

3. O mapa produzido na etapa 2 é convertido em uma [mensagem `Features`](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto#L85).

Neste notebook, você criará um dataset usando o NumPy.

Esse dataset terá 4 características:

- uma característica booleana, `False` ou `True` com a mesma probabilidade
- uma característica de número inteiro escolhida de maneira uniforme e aleatória de `[0, 5]`
- uma característica de string gerada a partir de uma tabela de string ao usar a característica de número inteiro como um índice
- uma característica float de uma distribuição normal padrão

Considere um exemplo que consiste em 10 mil observações distribuídas de maneira independente e idêntica de cada uma das distribuições acima:

In [ ]:
# The number of observations in the dataset.
n_observations = int(1e4)

# Boolean feature, encoded as False or True.
feature0 = np.random.choice([False, True], n_observations)

# Integer feature, random from 0 to 4.
feature1 = np.random.randint(0, 5, n_observations)

# String feature.
strings = np.array([b'cat', b'dog', b'chicken', b'horse', b'goat'])
feature2 = strings[feature1]

# Float feature, from a standard normal distribution.
feature3 = np.random.randn(n_observations)

Cada um desses atributos pode ser coagido em um tipo compatível com `tf.train.Example` usando um `_bytes_feature`, `_float_feature` ou `_int64_feature`. Depois, é possível criar uma mensagem `tf.train.Example` a partir dessas características codificadas:

In [ ]:
def serialize_example(feature0, feature1, feature2, feature3):
  """
  Creates a tf.train.Example message ready to be written to a file.
  """
  # Create a dictionary mapping the feature name to the tf.train.Example-compatible
  # data type.
  feature = {
      'feature0': _int64_feature(feature0),
      'feature1': _int64_feature(feature1),
      'feature2': _bytes_feature(feature2),
      'feature3': _float_feature(feature3),
  }

  # Create a Features message using tf.train.Example.

  example_proto = tf.train.Example(features=tf.train.Features(feature=feature))
  return example_proto.SerializeToString()

Por exemplo, suponha que você tenha uma única observação do dataset, `[False, 4, bytes('goat'), 0.9876]`. Você pode criar e imprimir a mensagem `tf.train.Example` para essa observação usando `create_message()`. Cada observação será escrita como uma mensagem `Features` conforme acima. Observe que a [mensagem](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/example.proto#L88) `tf.train.Example` é apenas um wrapper da mensagem `Features`:

In [ ]:
# This is an example observation from the dataset.

example_observation = []

serialized_example = serialize_example(False, 4, b'goat', 0.9876)
serialized_example

Para decodificar a mensagem, use o método `tf.train.Example.FromString`.

In [ ]:
example_proto = tf.train.Example.FromString(serialized_example)
example_proto

## Detalhes do formato TFRecords

Um arquivo TFRecord contém uma sequência de registros. O arquivo só pode ser lido sequencialmente.

Cada registro contém uma string de bytes, para a carga útil de dados, além do comprimento de dados, e hashes CRC-32C ([CRC de 32 bits](https://en.wikipedia.org/wiki/Cyclic_redundancy_check#CRC-32_algorithm) usando os hashes [polinomiais Castagnoli](https://en.wikipedia.org/wiki/Cyclic_redundancy_check#Standards_and_common_use)) para a verificação da integridade.

Cada registro é armazenado nos seguintes formatos:

```
uint64 length
uint32 masked_crc32_of_length
byte   data[length]
uint32 masked_crc32_of_data
```

Os registros são concatenados para produzir um arquivo. Os CRCs estão [descritos aqui](https://en.wikipedia.org/wiki/Cyclic_redundancy_check), e a máscara de um CRC é:

```
masked_crc = ((crc &gt;&gt; 15) | (crc &lt;&lt; 17)) + 0xa282ead8ul
```


Observação: não há nenhum requisito para usar o `tf.train.Example` nos arquivos TFRecord. `tf.train.Example` é apenas um método de serialização de dicionários para strings de bytes. Qualquer string de bytes que possa ser decodificada no TensorFlow pode ser armazenada em um arquivo TFRecord. Exemplos incluem: linhas de texto, JSON (usando `tf.io.decode_json_example`), dados de imagens codificados ou `tf.Tensors` serializados (usando `tf.io.serialize_tensor`/`tf.io.parse_tensor`). Veja mais opções no módulo `tf.io`.

## Arquivos TFRecord usando `tf.data`

O módulo `tf.data` também fornece ferramentas para ler e escrever dados no TensorFlow.

### Escrevendo um arquivo TFRecord

A maneira mais fácil de colocar os dados em um dataset é usando o método `from_tensor_slices`.

Aplicado a um array, ele retorna um dataset de escalares:

In [ ]:
tf.data.Dataset.from_tensor_slices(feature1)

Aplicado a uma tupla de arrays, ele retorna um dataset de tuplas:

In [ ]:
features_dataset = tf.data.Dataset.from_tensor_slices((feature0, feature1, feature2, feature3))
features_dataset

In [ ]:
# Use `take(1)` to only pull one example from the dataset.
for f0,f1,f2,f3 in features_dataset.take(1):
  print(f0)
  print(f1)
  print(f2)
  print(f3)

Use o método `tf.data.Dataset.map` para aplicar uma função a cada elemento de um `Dataset`.

A função mapeada precisa operar no modo grafo do TensorFlow — ela precisa operar com e retornar `tf.Tensors`. Uma função que não seja de tensores, como `serialize_example`, pode ser empacotada com `tf.py_function` para torná-la compatível.

Ao usar `tf.py_function`, é necessário especificar as informações de formato e tipo indisponíveis de outra maneira:

In [ ]:
def tf_serialize_example(f0,f1,f2,f3):
  tf_string = tf.py_function(
    serialize_example,
    (f0, f1, f2, f3),  # Pass these args to the above function.
    tf.string)      # The return type is `tf.string`.
  return tf.reshape(tf_string, ()) # The result is a scalar.

In [ ]:
tf_serialize_example(f0, f1, f2, f3)

Aplique essa função a cada elemento do dataset:

In [ ]:
serialized_features_dataset = features_dataset.map(tf_serialize_example)
serialized_features_dataset

In [ ]:
def generator():
  for features in features_dataset:
    yield serialize_example(*features)

In [ ]:
serialized_features_dataset = tf.data.Dataset.from_generator(
    generator, output_types=tf.string, output_shapes=())

In [ ]:
serialized_features_dataset

E os escreva em um arquivo TFRecord:

In [ ]:
filename = 'test.tfrecord'
writer = tf.data.experimental.TFRecordWriter(filename)
writer.write(serialized_features_dataset)

### Lendo um arquivo TFRecord

Você também pode ler o arquivo TFRecord usando a classe `tf.data.TFRecordDataset`.

Encontre mais informações sobre como consumir arquivos TFRecord usando o `tf.data` no guia [tf.data: crie pipelines de entrada do TensorFlow](https://www.tensorflow.org/guide/data#consuming_tfrecord_data).

Usar `TFRecordDataset`s pode ser útil para padronizar os dados de entrada e otimizar o desempenho.

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

Neste ponto, o dataset contém mensagens `tf.train.Example` serializadas. Quando elas são iteradas, retornam tensores de strings escalares.

Use o método `.take` para mostrar apenas os 10 primeiros registros.

Observação: a iteração de `tf.data.Dataset` só funciona com a eager execution ativada.

In [ ]:
for raw_record in raw_dataset.take(10):
  print(repr(raw_record))

Esses tensores podem ser processados usando a função abaixo. Observe que `feature_description` é necessário porque `tf.data.Dataset`s usam a execução de grafo e precisam dessa descrição para criar a assinatura de formato e tipo:

In [ ]:
# Create a description of the features.
feature_description = {
    'feature0': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature1': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'feature2': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'feature3': tf.io.FixedLenFeature([], tf.float32, default_value=0.0),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

Como alternativa, use `tf.parse_example` para processar o lote inteiro de uma vez. Aplique essa função a cada item no dataset usando o método `tf.data.Dataset.map`:

In [ ]:
parsed_dataset = raw_dataset.map(_parse_function)
parsed_dataset

Use a eager execution para mostrar as observações no dataset. Há 10.000 observações nesse dataset, mas você só mostrará as 10 primeiras. Os dados são exibidos como um dicionário de características. Cada item é um `tf.Tensor`, e o elemento `numpy` desse tensor mostra o valor da característica:

In [ ]:
for parsed_record in parsed_dataset.take(10):
  print(repr(parsed_record))

Aqui, a função `tf.parse_example` descompacta os campos `tf.train.Example` em tensores padrão.

## Arquivos TFRecord em Python

O módulo `tf.io` também contém funções puras do Python para ler e escrever arquivos TFRecord.

### Escrevendo um arquivo TFRecord

Em seguida, escreva as 10.000 observações no arquivo `test.tfrecord`. Cada observação é convertida em uma mensagem `tf.train.Example` e depois escrita no arquivo. Em seguida, você pode verificar se o arquivo `test.tfrecord` foi criado:

In [ ]:
# Write the `tf.train.Example` observations to the file.
with tf.io.TFRecordWriter(filename) as writer:
  for i in range(n_observations):
    example = serialize_example(feature0[i], feature1[i], feature2[i], feature3[i])
    writer.write(example)

In [ ]:
!du -sh {filename}

### Lendo um arquivo TFRecord

Esses tensores serializados podem ser processados facilmente usando `tf.train.Example.ParseFromString`:

In [ ]:
filenames = [filename]
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset

In [ ]:
for raw_record in raw_dataset.take(1):
  example = tf.train.Example()
  example.ParseFromString(raw_record.numpy())
  print(example)

Isso retorna um proto `tf.train.Example` que é difícil de ser usado no seu estado, mas é basicamente uma representação de:

```
Dict[str,
     Union[List[float],
           List[int],
           List[str]]]
```

O código a seguir converte manualmente o `Example` em um dicionário de arrays do NumPy, sem usar o TensorFlow Ops. Confira mais detalhes [no arquivo PROTO](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/example/feature.proto).

In [ ]:
result = {}
# example.features.feature is the dictionary
for key, feature in example.features.feature.items():
  # The values are the Feature objects which contain a `kind` which contains:
  # one of three fields: bytes_list, float_list, int64_list

  kind = feature.WhichOneof('kind')
  result[key] = np.array(getattr(feature, kind).value)

result

## Tutorial: como ler e escrever os dados de imagem

Este é um exemplo de como ler e escrever dados de imagem usando TFRecords do começo ao fim. Usando uma imagem como dados de entrada, você escreverá os dados como um arquivo TFRecord, lerá o arquivo novamente e exibirá a imagem.

Isso pode ser útil se, por exemplo, você quiser usar vários modelos no mesmo dataset de entrada. Em vez de armazenar os dados brutos da imagem, eles podem ser processados no formato TFRecords, que pode ser usado em processamento e modelagem adicionais.

Primeiro, vamos baixar [esta imagem](https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg) de um gato na neve e [esta foto](https://upload.wikimedia.org/wikipedia/commons/f/fe/New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg) da Ponte Williamsburg, em Nova York, sendo construída.

### Busque as imagens

In [ ]:
cat_in_snow  = tf.keras.utils.get_file(
    '320px-Felis_catus-cat_on_snow.jpg',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/320px-Felis_catus-cat_on_snow.jpg')

williamsburg_bridge = tf.keras.utils.get_file(
    '194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/194px-New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg')

In [ ]:
display.display(display.Image(filename=cat_in_snow))
display.display(display.HTML('Image cc-by: <a "href=https://commons.wikimedia.org/wiki/File:Felis_catus-cat_on_snow.jpg">Von.grzanka</a>'))

In [ ]:
display.display(display.Image(filename=williamsburg_bridge))
display.display(display.HTML('<a "href=https://commons.wikimedia.org/wiki/File:New_East_River_Bridge_from_Brooklyn_det.4a09796u.jpg">From Wikimedia</a>'))

### Escreva o arquivo TFRecord

Como antes, codifique as características como tipos compatíveis com `tf.train.Example`. Isso armazena a característica da string de imagem bruta, além da altura, largura, profundidade e da característica `label` arbitrária. A última é usada quando você escreve o arquivo para distinguir entre a imagem de um gato e a imagem de uma ponte. Use `0` para a imagem do gato e `1` para a imagem da ponte:

In [ ]:
image_labels = {
    cat_in_snow : 0,
    williamsburg_bridge : 1,
}

In [ ]:
# This is an example, just using the cat image.
image_string = open(cat_in_snow, 'rb').read()

label = image_labels[cat_in_snow]

# Create a dictionary with features that may be relevant.
def image_example(image_string, label):
  image_shape = tf.io.decode_jpeg(image_string).shape

  feature = {
      'height': _int64_feature(image_shape[0]),
      'width': _int64_feature(image_shape[1]),
      'depth': _int64_feature(image_shape[2]),
      'label': _int64_feature(label),
      'image_raw': _bytes_feature(image_string),
  }

  return tf.train.Example(features=tf.train.Features(feature=feature))

for line in str(image_example(image_string, label)).split('\n')[:15]:
  print(line)
print('...')

Observe que todas as características estão agora armazenadas na mensagem `tf.train.Example`. Em seguida, funcionalize o código acima e escreva as mensagens de exemplo para um arquivo chamado `images.tfrecords`:

In [ ]:
# Write the raw image files to `images.tfrecords`.
# First, process the two images into `tf.train.Example` messages.
# Then, write to a `.tfrecords` file.
record_file = 'images.tfrecords'
with tf.io.TFRecordWriter(record_file) as writer:
  for filename, label in image_labels.items():
    image_string = open(filename, 'rb').read()
    tf_example = image_example(image_string, label)
    writer.write(tf_example.SerializeToString())

In [ ]:
!du -sh {record_file}

### Leia o arquivo TFRecord

Agora você tem o arquivo — `images.tfrecords` — e pode iterar os registros nele para ler o que você escreveu. Considerando que, nesse exemplo, você só reproduzirá a imagem, a única característica de que você precisará é a string de imagem bruta. Faça a extração dela usando os getters descritos acima, especificamente `example.features.feature['image_raw'].bytes_list.value[0]`. Você também pode usar os rótulos para determinar qual registro é o gato e qual é a ponte:

In [ ]:
raw_image_dataset = tf.data.TFRecordDataset('images.tfrecords')

# Create a dictionary describing the features.
image_feature_description = {
    'height': tf.io.FixedLenFeature([], tf.int64),
    'width': tf.io.FixedLenFeature([], tf.int64),
    'depth': tf.io.FixedLenFeature([], tf.int64),
    'label': tf.io.FixedLenFeature([], tf.int64),
    'image_raw': tf.io.FixedLenFeature([], tf.string),
}

def _parse_image_function(example_proto):
  # Parse the input tf.train.Example proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, image_feature_description)

parsed_image_dataset = raw_image_dataset.map(_parse_image_function)
parsed_image_dataset

Recupere as imagens do arquivo TFRecord:

In [ ]:
for image_features in parsed_image_dataset:
  image_raw = image_features['image_raw'].numpy()
  display.display(display.Image(data=image_raw))